# RAG Submissions Blender (Auto-tuning)

Ноутбук для **автоматического подбора правил ансамблирования (blending)** RAG-сабмитов по локальному ground truth.

Идея:

- есть два сабмита `submission_1.csv` и `submission_2.csv`;
- есть `ground_truth.csv` с колонками `id` и `answer`;
- мы перебираем набор правил (пороги длины, похожести, дефолтный источник) и
  выбираем конфиг, который даёт **максимальный mean token-F1** на ground truth;
- затем этим лучшим конфигом строим финальный `submission_blended.csv`.


## Block 0. Конфиг и импорты

In [ ]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import difflib
import re

BASE_DIR = Path('.').resolve()
INPUT_DIR = BASE_DIR / 'inputs'
OUTPUT_DIR = BASE_DIR / 'outputs'

INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Пути к сабмитам и ground truth
SUB1_PATH = INPUT_DIR / 'submission_1.csv'
SUB2_PATH = INPUT_DIR / 'submission_2.csv'
GT_PATH   = INPUT_DIR / 'ground_truth.csv'

# Конфигурация колонок
ID_COL = 'id'
ANSWER_COL = 'answer'
REFS_COL = 'refs_json'  # опционально

print('BASE_DIR  :', BASE_DIR)
print('INPUT_DIR :', INPUT_DIR)
print('OUTPUT_DIR:', OUTPUT_DIR)


## Block 1. Загрузка сабмитов и ground truth

In [ ]:
def load_submission(path: Path, name: str) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f'Не найден сабмит {name}: {path}')
    df = pd.read_csv(path)
    print(f'{name}: shape =', df.shape)
    if ID_COL not in df.columns or ANSWER_COL not in df.columns:
        raise KeyError(f'В сабмите {name} должны быть колонки {ID_COL!r} и {ANSWER_COL!r}')
    return df

sub1 = load_submission(SUB1_PATH, 'sub1')
sub2 = load_submission(SUB2_PATH, 'sub2')

if not GT_PATH.exists():
    raise FileNotFoundError(f'Не найден ground truth: {GT_PATH}')
gt = pd.read_csv(GT_PATH)
print('ground truth shape =', gt.shape)
if ID_COL not in gt.columns or ANSWER_COL not in gt.columns:
    raise KeyError(f'В ground truth должны быть колонки {ID_COL!r} и {ANSWER_COL!r}')

# ограничимся только общими id, которые есть во всех трёх таблицах
ids1 = set(sub1[ID_COL])
ids2 = set(sub2[ID_COL])
ids_gt = set(gt[ID_COL])
common_ids = sorted(list(ids1 & ids2 & ids_gt))
print('\nОбщих id во всех трёх:', len(common_ids))

sub1c = sub1[sub1[ID_COL].isin(common_ids)].copy()
sub2c = sub2[sub2[ID_COL].isin(common_ids)].copy()
gtc   = gt[gt[ID_COL].isin(common_ids)].copy()

sub1c.sort_values(ID_COL, inplace=True)
sub2c.sort_values(ID_COL, inplace=True)
gtc.sort_values(ID_COL, inplace=True)

blend_df = pd.DataFrame({ID_COL: common_ids})
blend_df['answer_1'] = sub1c[ANSWER_COL].astype(str).fillna('')
blend_df['answer_2'] = sub2c[ANSWER_COL].astype(str).fillna('')
blend_df['gt_answer'] = gtc[ANSWER_COL].astype(str).fillna('')

if REFS_COL in sub1c.columns and REFS_COL in sub2c.columns:
    blend_df['refs_1'] = sub1c[REFS_COL].astype(str).fillna('')
    blend_df['refs_2'] = sub2c[REFS_COL].astype(str).fillna('')
else:
    blend_df['refs_1'] = ''
    blend_df['refs_2'] = ''

print('\nblend_df preview:')
blend_df.head()


## Block 2. Вспомогательные функции: токенизация, F1, похожесть, валидность

In [ ]:
def tokenize(text: str) -> List[str]:
    if text is None:
        return []
    return re.findall(r"\w+", str(text).lower(), flags=re.UNICODE)

def simple_f1(pred: str, gt: str) -> float:
    pred_tokens = tokenize(pred)
    gt_tokens = tokenize(gt)
    if not pred_tokens or not gt_tokens:
        return 0.0
    common = set(pred_tokens) & set(gt_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

def simple_similarity(a: str, b: str) -> float:
    a = (a or '').strip()
    b = (b or '').strip()
    if not a and not b:
        return 1.0
    if not a or not b:
        return 0.0
    return difflib.SequenceMatcher(None, a, b).ratio()

def jaccard_tokens(a: str, b: str) -> float:
    ta = set(tokenize(a))
    tb = set(tokenize(b))
    if not ta and not tb:
        return 1.0
    if not ta or not tb:
        return 0.0
    inter = len(ta & tb)
    union = len(ta | tb)
    if union == 0:
        return 0.0
    return inter / union

def is_invalid_answer(text: str) -> bool:
    if text is None:
        return True
    t = str(text).strip()
    if not t:
        return True
    lowered = t.lower()
    if lowered.startswith('error:') or 'traceback' in lowered:
        return True
    return False

def answer_length(text: str) -> int:
    if text is None:
        return 0
    return len(str(text))


## Block 3. Правило blending, зависящее от параметров

Мы определяем функцию, которая выбирает ответ на основе набора **параметров**:

- `min_len` — минимальная «нормальная» длина ответа;
- `max_len` — максимальная разумная длина (простыни выше этого считаем подозрительными);
- `sim_thr` — порог похожести для "почти одинаковых" ответов;
- `default_src` — какой сабмит считать более сильным по умолчанию (`1` или `2`).

In [ ]:
def blend_two_answers_with_params(a1: str, a2: str, params: Dict[str, Any]) -> Dict[str, Any]:
    a1 = str(a1 or '')
    a2 = str(a2 or '')

    min_len = params.get('min_len', 15)
    max_len = params.get('max_len', 800)
    sim_thr = params.get('sim_thr', 0.9)
    default_src = params.get('default_src', 2)  # 1 или 2

    invalid_1 = is_invalid_answer(a1)
    invalid_2 = is_invalid_answer(a2)

    len1 = answer_length(a1)
    len2 = answer_length(a2)

    sim_seq = simple_similarity(a1, a2)
    sim_jac = jaccard_tokens(a1, a2)

    # 1) один невалидный → другой
    if invalid_1 and not invalid_2:
        return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}
    if invalid_2 and not invalid_1:
        return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 2) оба невалидные → первый
    if invalid_1 and invalid_2:
        return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 3) короткие vs нормальные
    if len1 < min_len <= len2:
        return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}
    if len2 < min_len <= len1:
        return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 4) слишком длинные
    if len1 > max_len and len2 <= max_len:
        return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}
    if len2 > max_len and len1 <= max_len:
        return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 5) почти одинаковые → берём более короткий
    if sim_seq >= sim_thr or sim_jac >= sim_thr:
        if len1 <= len2:
            return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}
        else:
            return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 6) default: доверяем default_src
    if default_src == 1:
        return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}
    else:
        return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}


## Block 4. Функция оценки качества для заданных параметров

In [ ]:
def evaluate_params(params: Dict[str, Any], sample_size: int = None) -> Dict[str, Any]:
    df = blend_df
    if sample_size is not None and sample_size < len(df):
        df = df.sample(sample_size, random_state=42)

    f1_list = []
    for _, row in df.iterrows():
        a1 = row['answer_1']
        a2 = row['answer_2']
        gt = row['gt_answer']
        res = blend_two_answers_with_params(a1, a2, params)
        f1 = simple_f1(res['answer'], gt)
        f1_list.append(f1)

    mean_f1 = float(np.mean(f1_list)) if f1_list else 0.0
    out = params.copy()
    out['mean_f1'] = mean_f1
    out['n'] = len(df)
    return out


## Block 5. Перебор конфигураций (grid search)

Сделаем простой grid search по:

- `min_len`  ∈ {10, 20, 40};
- `max_len`  ∈ {500, 800, 1200};
- `sim_thr`  ∈ {0.85, 0.9, 0.95};
- `default_src` ∈ {1, 2}.

При желании можно расширить/сузить сетку. Для скорости можно задать `sample_size`.

In [ ]:
min_len_options = [10, 20, 40]
max_len_options = [500, 800, 1200]
sim_thr_options = [0.85, 0.9, 0.95]
default_src_options = [1, 2]

# Можно ограничить размер выборки для быстрого подбора (None = вся выборка)
SAMPLE_SIZE_FOR_SEARCH = None  # например, 200

results = []
for min_len in min_len_options:
    for max_len in max_len_options:
        for sim_thr in sim_thr_options:
            for default_src in default_src_options:
                params = {
                    'min_len': min_len,
                    'max_len': max_len,
                    'sim_thr': sim_thr,
                    'default_src': default_src,
                }
                res = evaluate_params(params, sample_size=SAMPLE_SIZE_FOR_SEARCH)
                results.append(res)

search_df = pd.DataFrame(results)
search_df = search_df.sort_values('mean_f1', ascending=False).reset_index(drop=True)
print('Лучшие конфиги по mean_f1:')
search_df.head(10)


## Block 6. Выбор лучшего конфига и blending на всех id

In [ ]:
best_params = search_df.iloc[0][['min_len', 'max_len', 'sim_thr', 'default_src']].to_dict()
print('Лучший конфиг:', best_params)

blend_results_full = []
for _, row in tqdm(blend_df.iterrows(), total=len(blend_df), desc='Blending with best params'):
    a1 = row['answer_1']
    a2 = row['answer_2']
    res = blend_two_answers_with_params(a1, a2, best_params)
    blend_results_full.append(res)

blend_meta_full = pd.DataFrame(blend_results_full)
blend_meta_full.head()


In [ ]:
final_df = blend_df[[ID_COL]].copy()
final_df[ANSWER_COL] = blend_meta_full['answer']
final_df['src'] = blend_meta_full['src']
final_df['sim_seq'] = blend_meta_full['sim_seq']
final_df['sim_jac'] = blend_meta_full['sim_jac']

# выбираем refs_json в зависимости от src
final_refs = []
for i, row in final_df.iterrows():
    src = row['src']
    if src == 1:
        final_refs.append(blend_df.loc[i, 'refs_1'])
    elif src == 2:
        final_refs.append(blend_df.loc[i, 'refs_2'])
    else:
        final_refs.append('')

final_df[REFS_COL] = final_refs

print('final_df shape:', final_df.shape)
final_df.head()


## Block 7. Сравнение F1 сабмитов и blended-версии на всём ground truth

In [ ]:
f1_1_all = []
f1_2_all = []
f1_b_all = []

for _, row in blend_df.iterrows():
    gt_ans = row['gt_answer']
    f1_1_all.append(simple_f1(row['answer_1'], gt_ans))
    f1_2_all.append(simple_f1(row['answer_2'], gt_ans))

# final_df соответствует blend_df по индексу
for i, row in final_df.iterrows():
    gt_ans = blend_df.loc[i, 'gt_answer']
    f1_b_all.append(simple_f1(row[ANSWER_COL], gt_ans))

print('=== Итоговая оценка по token-F1 (на общих id) ===')
print('sub1   mean F1:', float(np.mean(f1_1_all)))
print('sub2   mean F1:', float(np.mean(f1_2_all)))
print('blend  mean F1:', float(np.mean(f1_b_all)))


## Block 8. Сохранение blended-сабмита

In [ ]:
BLEND_PATH = OUTPUT_DIR / 'submission_blended_auto.csv'

cols_for_submit = [ID_COL, ANSWER_COL]
if REFS_COL in final_df.columns:
    cols_for_submit.append(REFS_COL)

submit_blend = final_df[cols_for_submit].copy()
submit_blend.to_csv(BLEND_PATH, index=False)
print('Авто-бленд сабмит сохранён в:', BLEND_PATH)
submit_blend.head()


## Итог

В этом ноутбуке:

1. Загрузили два сабмита и ground truth.
2. Привели их к единой таблице `blend_df` с `answer_1`, `answer_2`, `gt_answer`.
3. Определили параметризованное правило blending (`min_len`, `max_len`, `sim_thr`, `default_src`).
4. Сделали grid search по конфигам и выбрали лучший по mean token-F1.
5. Этим конфигом собрали финальный blended-сабмит.
6. Сравнили F1 `sub1`, `sub2` и blended-версии.
7. Сохранили итоговый `submission_blended_auto.csv`.

На финале можно:
- запускать этот ноутбук на валидационной части (где есть GT) для подбора параметров;
- переносить найденный конфиг (значения параметров) в основной блендер, который работает уже без GT.